<u>Avertissement</u> : Ce rapport a été rédigé depuis un carnet jupyter lab. Il est également compatible sous jupyter notebook. Pour l'exécuter correctemnt, il faut lancer un jupyter notebook depuis le répertoire principal de ce projet : cs_project_lasme_nathane pour s'assurer que tous les liens hypertextes du notebook sont respectés. Cela est nécessaire pour l'affichage des images, logos et les navigations entre les codes sources de la section implémentation SDP. Ce rapport n'a pas été conçu pour Google Colab et ne nous sommes pas sûrs de garantir la bonne exécution des liens hypertextes sur Colab.

# <center> PROJET DE COMPRESSED SENSING </center>

<center> <img src = "img/logo_ensae.png" height="300" width="300"/> </center>

#### <center>  Réalisé par : </center>

<center>BERREBI Nathane</center>
<center>ESSOH Lasme Ephrem Dominique </center>

<br>

<center>Elèves-Ingénieurs 3A DSSA (2021 - 2022)</center>




#### <center> Thème :  </center> 

# <center>Semidefinite relaxations for certifying robustness to adversarial examples</center>

Ce travail s'inscrit dans le cadre du cours de Compressed Sensing. Il porte sur le Papier de Aditi Raghunathan, Jacob Steinhardt et Percy Liang présenté au NeurIPS en 2018. Dans ce papier, il s'agit essentiellement d'apporter une solution à l'échec de prédiction des réseaux de neurones en face "d'adversarial examples", la solution proposée étant basée sur la relaxation convexe SDP.

# Introduction

Au cours de cette dernière décénie, pas un seul jour ne passe sans que les prouesses des réseaux de neurones artificiels soient mises en évidences que ce soit à travers les médias qu'à travers la prolifération de publications scientifiques sur le sujet. Ces réseaux sont à la base de la révolution de l'Intelligence Artificielle surtout dans le domaine de la vision par ordinateur. Cependant,  malgré leurs performances impressionnantes dans diverses tâches relatives à la vision, les réseaux de neurones peinent, de manière catastrophique, à prédire correctement l'étiquette associée à une image si les pixels de celle-ci sont pertubés par un bruit de corruption imperceptible par ces réseaux de neurones. Ainsi, une image d'un Panda dont les pixels ont été pertubées pourrait être perçue comme étant celle d'un Gibbon alors qu'un être humaine perçevrait clairement un Panda : c'est la problématique d'un "adversarial example", c'est-à-dire celle d'exemple d’objet capable de tromper et déjouer un algorithme d’un réseau de neurones en lui faisant croire qu’il doivent être classifié en tant que tel objet alors que ce n’est pas le cas.
<br>

Pour résoudre ce problème des exemples contradictoires, la façon la plus simple et naturelle de procéder est de construire des réseaux entrainés sur des adversariales training en générant un certain nombre d’adversarial examples contre notre réseau de neurones puis à entraîner notre réseau de neurones sur ces données générées. Cette façon de procédé à conduit à "une course aux armements" entre les défenseurs qui tentent de former des réseaux robustes par les adversariales learning et les attaquants qui essaient de construire des exemples adverses nouveaux biaisant ces réseaux robustes. Ainsi, au risque de proposer un moyen efficace de se défendre contre les exemples contradictoires, cette courses aux armements peut être sans fin.

<br>

L'une des stratégie pour mettre fin à cette course aux armements est de développer des "défenses certifiées", c'est-à-dire des défenses dont la robustesse contre tous les attaquants d'une certaine famille est prouvée. Ces défenses certifiées sont basées sur des relaxations convexes qui construisent une limite supérieure sur la perte la plus défavorable pour tous les attaquants de la famille. 

<br>
Dans leur article, Raghunathan et al. proposent une nouvelle relaxation : la relaxion SDP pour certifier la robustesse des réseaux de neurones multicouches avec fonction d'activation ReLU.

<br> Notre travail, basé sur l'article de Raghunathan et al.(2018), décrit comment appliquer une relaxation dans le cadre du jeu de données MNIST et en présence d'adversarials examples.

# Aversarials examples

<b>Définition : </b> Un “adversarial example” ou exemple contradictoire est un exemple d’objet capable de tromper et déjouer un algorithme d’un réseau de neurones en lui faisant croire qu’il doit être classifié en tant que tel objet alors que ce n’est pas le cas. 

<br> 

<b> Caractérisation </b> : Un “avdersarial example” est un ensemble de données correctement initialisées auxquelles on aurait ajouté une perturbation imperceptible par le réseau de neurones afin d’entraîner une mauvaise classification.

<br> 

<b> Exemple (Jun Yan, 2019) : </b> Prenons l'exemple de la classification d’images sur les pandas qu’un réseau de neurones reconnaît correctement comme un panda avec un taux de confiance de 57,7 %. Si on ajoute un peu de perturbation soigneusement construite,  le même réseau de neurones classe maintenant l’image comme étant un gibbon avec une confiance de 99,3 % ! Il s’agit clairement d’une illusion d’optique, mais seulement pour le réseau de neurones. Nous, humains, pouvons clairement affirmer que ces deux images correspondent bien à des pandas. En fait, nous ne pouvons même pas percevoir qu’un peu de perturbation a été ajoutée à l’image originale de gauche pour construire l’adversarial example à droite !

<center> <img src = "img/adversarial_example.png" height="300" width="600"/> </center>

<b> Nous proposons une implémentation d'un adversarial example à travers ce notebook : <a href = "sources/generation_adv_example/generate_adv.ipynb">génération d'un exemple contraire.</a></b>

Les adversarials examples posent de sérieux problèmes qui pourraient se transposer dans notre quotidienne réelle. Par exemple, en ce qui concerne la conduite de véhicule autonome, une mauvaisse détection des panneaux de signalisation peut entraîner des conséquences dangereuses. En effet, Eykholt et al (2018) ont présenté comment des réseaux de neurones embarqués dans véhicules autonomes souffraient à détecter un panneau sur lequel on a collé des stickers ou réalisé des graffitis. Le problème a été illustré par les auteurs avec un panneau stop (image à gauche dont la représentation pertubée à droite) qui a été classifié comme étant une limitation de vitesse à 45 km/h !

<center> <img src = "img/adversarial_example_ac.png" height="200" width="400"/> </center>

# Certification de la robustesse d'un réseau de neurones 

Nous présentons dans la suite un moyen de lutter contre les adversarials examples pour des réseaux de neurones multicouche avec fonction d'activation ReLU pour la classification. Ainsi, cette section présente le cadre mathématique de la notion de certification d'un réseau de neurones qui se ramène à un problème d'optimisation non convexe.

### Notations 

Soit $z \in R^n$ un vecteur, nous désignons par $z_i$ sa $i-$ème composante.


Soit $Z \in R^{m \times n}$ une matrine à $m$ lignes et $n$ colonnes, nous désignons par $Z_i$ sa $i-$ème ligne.

Soit $f : R \rightarrow R$ une fonction réelle et $z$ un vecteur de $z \in R^n$, $f(z)$ désigne un vecteur de $z \in R^n$ dont les composantes sont les $f(z_i) = (f(z))_i$.

Pour $z,y ∈ R^n$, $z≽y$ désigne que $z_i ≥ yi$ pour $i=1,2,...,n$. 

Nous utiliserons la notation $z_1 ⊙ z_2$ pour représenter le produit élément par élément des deux vecteurs.

On définit $B_ε(\bar{x}) = \{x : ||x - \bar{x}||\infty ≤ε\}$ pour désigner la boule $L_\infty$ autour de $\bar{x}$. 

Nous désignons le vecteur dont toutes les composantes sont zéros par $\mathbb{0}$ et le vecteur dont toutes les composantes sont uns par $\mathbb{1}$.


### Réseaux neurones multicouches avec activation ReLU pour la classification

Ce travail se concentre sur les réseaux de neurones muticouches avec fonction d'activation ReLU pour la classification. Un tel réseau de neurones avec $L$ couches cachées est une fonction $f$ définie comme suit : 

 - soit $x^0 \in R^d$ appelé "input",

 - soit $x^1, \ldots, x^L$ appelés des "vecteurs d'activations" sur les couches cachées,

 - supposons que $m_i \in N$ est le nombre d'unités d'une couche $i$,

alors 


 - $x^i$ est lié à $x^{i-1}$ par la relation $x^i = max(W^{i-1}x^{i-1}, 0) = ReLU (W^{i-1}x^{i-1})$, où $ W^{i-1} \in R^{m_i \times m_{i-1}}$ sont les poids du réseau (on a omis les terme de biais),

<br>
 
 - et $f(x^0)$ est un vecteur de $R^k$, obtenu à travers les transformations successives de $x^0$ à $x^L$ suivant la relation de récurrence ci-dessus, tel que $f(x^0)_j = c_j^Tx^L$ represente le score de la classe $j$. l'étiquette $y$ à assigner à un input $x^0$ est donc la classe avec le plus gros score : $y = argmax_{j = 1, \ldots, k}f(x^0)_j$
 
 <center> <img src = "img/ff_nn.png" height="200" width="400"/> </center>

### Modèle de l'attaquant et certification de robustesse

Etant donnée un réseau de neuronnes multicouches avec activations ReLU, on suppose que l'univers des inputs est en présence d'un attaqueur $A$ qui prend un input correct $\bar{x} \in R^d$ et renvoie une version pertubée de cet input $A(\bar{x})$ qui est passée au réseau de neurones au lieu du vrai input. De ce fait un attaqueur est une application $A : R^d \rightarrow R^d$. Dans ce travail, nous nous intéressons aux attaquants qui sont bornés suivant la norme $L_{\infty}$ : $A(\bar{x}) \in B_{\varepsilon}(\bar{x})$ pour un certain $\varepsilon > 0$.

L'attaquant $A$ est correcte si pour une paire d'observation $(\bar{x}, \bar{y})$, la prédiction fournie par le réseau en $A(\bar{x})$ est différente de $\bar{y}$ : c'est-à-dire $f(A(\bar{x})) \ne \bar{y}$ ou de manière équivalente le score rapporté par le réseau en $A(\bar{x})$ pour n'importe quelle classe $y \ne \bar{y}$ excède le score rapporté par le réseau en $A(\bar{x})$ pour la classe $\bar{y}$, ce que l'on peut écrire $f(A(\bar{x}))_y > f(A(\bar{x}))_{\bar{y}}$ .

En supposant que l'attaquant connait le réseau, un bonne manière de limiter son impact est de s'intéresser à la pire penalité possible que l'attaqueur peut infliger au réseau quand il fournit une classe incorrecte $y$ au lieu de $\bar{y}$. Cette pénalité est en fait :

$$l^*_y(\bar{x}, \bar{y})  = max_{A(x) \in B_{\varepsilon}(\bar{x})} ( f(A(x))_y - f(A(x))_{\bar{y}} )$$


On dira alors qu'un réseau de neurones est cértifié robuste sur l'exemple $(\bar{x}, \bar{y})$ si $l^*_y(\bar{x}, \bar{y})<0$ pour tout $y \ne \bar{y}$.

### Formulation du problème en terme d'optimisation

Pour une classe $y$ fixée et pour une paire d'exemple $(\bar{x}, \bar{y})$, la pire pénalité $l^*_y(\bar{x}, \bar{y})$ d'un réseau de neurones $f$ avec des poids $W$ peut s'exprimer comme un problème d'optimisation. La variable de décision est l'input $A(x)$ que nous noterons sans perte de généralité $x^0$. La fonction objectif est $f(x^0)_y - f(x^0)_{\bar{y}} = (c_y - c_{\bar{y}})^Tx^L$ qui est maximiser, avec où $x^L$ est l'activation de la couche finale i.e. la dernière transformation succéssive de $x^0$ suivant les couches cachées à travers le réseau. 

En imposant les contraintes adéquates dictées par le réseau de neurones et les constraintes de l'attaqueur A, $l^*_y(\bar{x}, \bar{y})$ est donné par le programme suivant :

$$ l^*_y(\bar{x}, \bar{y}) = max_{x^0, \ldots, x^L} (c_y - c_{\bar{y}})^Tx^L $$

<center>Sous contrainte : $ReLU (W^{i-1}x^{i-1})$ pour $i = 1 \ldots L$ (contraintes du réseau)</center>
<center> $\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: x^i = ||x^0_j - \bar{x}_j||\infty ≤ε$ pour $j = 1 \ldots d$ (contraintes du modèle d'attaque)</center>

D'un point de vu informatique, calculer $l^*$ est difficile en général, c'est pourquoi nous proposons une relaxation de cet objectif.

### Relaxation du problème d'optimisation

Pour un réseau de neurones, calculer $l^*_y(\bar{x}, \bar{y})$ implique de résoudre un problème d'optimisation non convexe qui est en général intraitable. Nous étudions dans ici une relaxation convexe qui permet de calculer une borne supérieure de $l^*_y$ : $L_y(\bar{x}, \bar{y}) \ge l^*_y(\bar{x}, \bar{y})$. De ce fait, comme discuté précédemment lorsque $L_y(\bar{x}, \bar{y}) < 0$, nous avons une certification de la robustesse du réseau pour l'input $(\bar{x}, \bar{y})$.

#### Transformation du problème en un problème QCQP

La source de la non-convexité du programme d'optimisation défini dans la section précédente vient des contraintes ReLU du réseau. Considérons alors une contrainte ReLU de la forme $z = max(z,0)$. Cette contrainte peut s'exprimer de manière équivalente sous les trois contraintes suivantes : $(i) z \ge x, (ii) z \ge 0$ et $ (iii) z(z-x) = 0$. En effet, la contrainte $(iii)$ assure que $z$ est égale soit à 0 soit à $x$ et les deux autres contraintes $(i)$ et $(ii)$ s'assurent que $z$ est au moins plus grand que ces deux valeurs d'égalités. Cette reformulation de la contrainte ReLU nous permet de remplacer des contraintes non linéaires du problème d'optimisation par des contraintes linéaires et quadratiques, transformant ainsi le programme en un programme quadratique à contraintes quadratiques (<a href = "https://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program"> QCQP </a> en anglais). Les programmes d'optimisation QCQP sont en général NP-Hard. Ce sont des programmes qui entre dans la boite des classiques programmes d'optimisation qui peuvent se résoudre à l'aide d'une relaxation convexe SDP.

#### Rélaxation d'un réseau de neurones multicouches ReLU

##### <u>Cas d'un réseau de neurones à une seule couche ReLU</u> :

Nous montrons d'abord comment ce QCQP peut être relaxé en un programme SDP pour les réseaux à une couche cachée. La relaxation pour les couches multiples est une extension simple et est présentée à la fin de cette section.

Considérons un réseau de neurones avec un seule couche cachée de $m$ unités. Soit un input $x \in R^d$ du réseau. L'activation de la couche cachée sera notée $z \in R^m$. Elle est reliée à $x$ par la relation $z = ReLU(Wx)$ où $W \in R^{m \times d}$ sont les poids du réseau.

Supposons que nous notre input est borné i.e. nous avons $l, u \in R^d$ tels que $l_j \le x_j \le u_j$. Par exemple, dans le cas de notre modèle d'attaqueur $L_\infty$, nous avons $l = \bar{x} - \varepsilon \mathbb{1}$ et $l = \bar{x} =u + \varepsilon \mathbb{1}$ où $\bar{x}$ est un input correcte et non pertubée.

Nous nous intéressons ici à la maximisation de l'objectif : $f(x) = c^Tz$ où $c \in R^m$, $c = c_y - c_{\bar{y}}$.

Nous utilisons la transformation des contraintes ReLU en contraintes linéaires et quadratiques, ce qui nous d'écrire un QCQP. A ce QCQP nous rajoutons la contrainte que les inputs sont bornées : $l_j \le x_j \le u_j$, ce qui est équivalent à une contrainte quadratique $(x_j -l_j)(x_j -u_j)≤0$, que nous réécrivons comme suit $x^2_j ≤(l_j +u_j)x_j -l_ju_j$. Nous avons alors le QCQP suivant :


$$l^*_y(\bar{x}, \bar{y}) = f_{QCQP} =  max_{x,z} c^Tx^L $$

<center>Sous contrainte : $z \ge 0, z \ge Wx, z^2 = z ⊙ (Wx)$ (contraintes du réseau)</center>
<center> $\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:  x^2 ≤ (l+u)⊙x - l ⊙ u $  (contraintes du modèle d'attaque)</center>

<br>
Nous relaxons maintenant ce dernier programme QCQP non convexe ci-dessus en un SDP convexe. L'idée de base est d'introduire un nouvel ensemble de variables représentant tous les monômes linéaires et quadratiques en x et z ; les contraintes du programme ci-dessus peuvent alors être écrites comme des fonctions linéaires de ces nouvelles variables.

<br>
Par exemple, en posant $P = vv^T$ où $ v =
\begin{pmatrix}
1 \\[0.5mm]
x\\[0.5mm]
z\\
\end{pmatrix}
$. Ainsi $P =  \begin{pmatrix} 
1 & x & z \\ 
x & x^2 & xz \\ 
z & xz & z^2
\end{pmatrix}$ =  $\begin{pmatrix} 
P[1] & P[x^T] & P[z^T] \\ 
P[x] & P[xx^T] & P[xz^T] \\ 
P[z] & P[zx^T] & P[zz^T]
\end{pmatrix}$

Ainsi, la relaxation convexe du programme principal peut sécrire omme suit :

$$f_{SDP} =  max_{P} c^TP[z] $$

<center>Sous contrainte : $P[x] \ge 0, P[z] \ge WP[x], diag(P[zz^T]) = diag(WP[xz^T])$ (contraintes ReLU)</center>
<center> $\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:  diag(P[xx^T]) \le (l+u)⊙P[x] - l ⊙ u$  (contraintes du modèle d'attaque)</center>
<center> $\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:  P[1] = 1, P≽0$  (contraintes du matriciels)</center>

<br>
Notons que nous considérons l'ensemble des matrices $P$ telles que $P ≽ 0$. Cet ensemble est convexe et est un sur-ensemble de l'ensemble original non-convexe. Par conséquent, le programme ci-dessus est un programme SDP ci-dessus fournissant une relaxation du QCQP avec $f_{SDP} ≥ f_{QCQP}$ , fournissant une borne supérieure sur $l^*_y(\bar{x}, \bar{y})$ qui pourrait servir de certificat de robustesse. 

##### <u> Cas d'un réseau de neurones multicouches ReLU </u> :

La relaxation SDP pour évaluer la robustesse des réseaux multicouches est une généralisation directe de la relaxation présentée pour une couche cachée. Les interactions entre $x_{i-1}$ et $x_i$ selon la contrainte ReLU  sont analogues à l'interaction entre la couche d'entrée $x$ et la couche cachée $z$ pour le cas à une couche. Supposons que nous ayons des bornes $l^{i-1}$, $u^{i-1} \in R^{m_{i-1}}$ sur les entrées des unités ReLU de la couche i telles que $l^{i-1} ≤x^{i-1} ≤u^{i-1}$. En écrivant les contraintes pour chaque couche de manière itérative, on obtient le programme SDP suivant :

$$f^{SDP}_y(\bar{x}, \bar{y}) =  max_{P} (c_{y} - c_{\bar{y}} )^T P[x^L] $$

<center>Sous contrainte : pour  $ i =  1, \ldots, L$ </center>

<center>$P[x^i] \ge 0, P[x^i] \ge W^{i - 1}P[x^{i - 1}]$ </center>

<center>$\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:diag(P[x^i(x^i)^T]) = diag(WP[x^{i - 1}(x^i)T])$ (contraintes ReLU)</center>

<center> $\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \:\:\: diag(P[x^{i - 1}(x^{i - 1})^T]) \le (l^{i - 1}+u^{i - 1})⊙P[x^{i - 1}] - l^{i - 1} ⊙ u^{i - 1}$  (contraintes du modèle d'attaque)</center>
<center> $\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:  P[1] = 1, P≽0$  (contraintes du matriciels)</center>


# Expérimentation

Dans cette section nous présentatons comment les auteurs du papier que nous étudions ont évalué leur méthode de relaxation SDP.

Le réseau de neurones sur lequel ils travaillent est un réseau multicouches avec fonction d'activation ReLU destiné à classfier le célèbre de jeu de donnée MNIST (pour Modified ou Mixed National Institute of Standards and Technology, est une base de données de chiffres écrits à la main) pour une tâche de classification.

Sur MNIST, les auteurs évaluent leur procédure de certification robuste par relaxation SDP sur trois architectures de réseau de neurones différentes et par rapport à deux autres procédures de certification distinctes : la relaxation LP et la certification par la méthode dite du gradient traités dans le cadre de papiers anciens à la publication sur laquelle nous travaillons.

Les architectures de réseaux sont les suivantes : 

- L'arichtecture Grad-NN : c'est un réseau de neurones avec deux couches cachées de 500 unités chacunes et robuste à des adversarials examples suivant la procédure de certification du gradient. Ce réseau n'a pas été entrainé, ses poids ont été récupéré par les auteurs auprès de leur pairs qui ont inventé et entrainé ce réseau ;

- L'arichtecture LP-NN : c'est un réseau de neurones avec deux couches cachées de 500 unités chacunes et robuste à des adversarials examples suivant la procédure de certification par la relaxtion LP. Ce réseau n'a, aussi, pas été entrainé, ses poids ont été récupéré par les auteurs auprès de leur pairs qui ont inventé et entrainé ce réseau ;

- L'architecture PGD-NN : C'est un réseau de neurones entièrement connecté avec quatre couches contenant 200, 100 et 50 unités entrâiné en utilisant des adversarials examples contre l'attaquant de déscente de gradient projété. Ce réseau a été entrainé par les auteurs de sorte à ce qu'il minimise une combinaison pondérée de la perte d'entropie croisée régulière et de la perte adversariale. 



Les procédures de certifications : 

- SDP-cert. Il s'agit du certificat que les auteurs ont proposé : la certfication par relaxation SDP.

- LP-cert. Il s'agit d'une méthode de certification basée sur la relaxation LP traitée dans la litterature précédente.

- Grad-cert. Il s'agit d'une méthode certification basée sur la norme maximale du gradient des prédictions du réseau.


Résultats : 

Le tableau ci-dessous présente les performances des trois différentes procédures de certification sur les trois réseaux. Pour chaque méthode de certification et chaque réseau, les auteurs évaluent les limites supérieures de $l^*$ (pire pénalité infligé par un attaqueur au réseau) sur 1000 mêmes images, choisi aléatoirement, dans le jeu de données de test de MNIST. A partir des limités supérieurs, les auteurs indiquent la fraction d'images qui n'ont pas été certifiés.

 <center> <img src = "img/results.png" height="1000" width="600"/> </center>

A partir de ce tableau, nous observons que SDP-cert est systématiquement plus performant que LP-cert et Grad-cert pour les trois réseaux. Grad-cert et LP-cert fournissent des certificats "vides" (erreur > 90%) sur des réseaux qui ne sont pas entraînés à minimiser ces certificats. 

# Implémentation SDP

En ce qui concerne l'implémentation du problème traité dans ce papier, nous nous sommes reférés des à sources relatifs au papier que nous avons trouvé sur internet. Nous avons globalement mis à jour les lignes de codes non fonctionnelles notamment les lignes de codes fonctions avec  tensorflow sous sa version 1 alors que nous sommes aujourd'hui à la version 2 et plusieurs changement ont été apportés dans l'API. Pour la réplication des résultats des auteurs il faut disposer de l'ensemble des codes sources, en Python et en Matlab, qui se trouvent dans le répertoire : <a href = "http://localhost:8888/lab/tree/sources/"> Fichiers sources</a> (/sources). 
    
Ce répertoire est organisé de la manière suivante : 

- *code* : contient tous les sources pour l'entrainement ou le transfert du réseau avec les différentes certifications. En particulier, il contient le fichier matlab - <a href = "http://localhost:8888/lab/tree/sources/code/matlab_sdp.m/">  matlab_sdp.m </a> (/sources/codes/matlab_sdp.m) - qui implémente la relaxation SDP sur des réseaux multicouches. C'est ce fichier qui communique avec Python via le module de matlab en Python : *matlab.engines* pour caculer les limites supérieures de certification. Notons que matlab.engines nécessite l'installation de matlab sur la machine qui l'exécute ; 


    
- *mnist_permuted_data* : contient les 1000 images annotées pour lesquelles une certification de robustesse est appliquée ;


- *models* : contient les fichiers de sauvegarde tensorflow relatifs aux trois différentes architectures de réseaux de neurones mentionnées précédemment. 


L'ensemble des répertoires sont liés et au fichiers source principale :  <a href = "http://localhost:8888/lab/tree/sources/code/certify.py"> certify.py</a> (/sources/codes/certify.py). C'est ce fichier qu'il faut exécuter depuis un terminal Python dans un environnement de programmation où sont installés les modules suivants : *numpy, matplotlib, scikit-learn, tensorflow et matlab.engines*.

# Conclusion

En conclusion, dans leur article, Raghunathan et al. (2018) proposent une nouvelle relaxation : la relaxion SDP pour certifier la robustesse qui s'applique aux réseaux ReLU pour la classification. Ils montrent que la relaxation qu’ils proposent est plus meilleure que les relaxations précédentes à leur papier et qu'elle produit des garanties de robustesse significatives sur trois réseaux étrangers différents dont les objectifs de formation sont agnostiques par rapport à la relaxation qu’ils proposent.